<a href="https://colab.research.google.com/github/ttimoff/A-B-test-project2/blob/main/%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%82_%D0%BF%D0%BE_AB_%D1%82%D0%B5%D1%81%D1%82%D0%B0%D0%BC_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Шаблон для проекта по A/B - тестам.

### `chat_id ` из Телеграма:

In [ ]:
chat_id = 1379613676

------------------

##  <ins>Задание 1.</ins>

### 0. Запосить у бота в Телеграме условие командой `/get_project1_desc`.

### Контекст
В связи с постановлением Центрального Банка
наше текущее содержание диалога с клиентом (скрипт)
операторов считается слишком жёстким,
и нам нужно придумать более мягкий скрипт,
который будет всё ещё иметь хорошие показатели.\
У нас есть два варианта более мягкого скрипта,
и мы хотим выбрать лучший из них.\
При этом у нас нет времени 
чтобы ждать вызревания долгих метрик вроде PV.
        
### Тест
Мы проводим тест,
в рамках которого мы разбиваем весь поток
на две части рановероятно,
где одной части мы продаём одним вариантом скрипта,
другой - другим.\
Целью этого теста
является поиск наилучшего скрипта.

### Статистические вводные
Уровень значимости 5%.\
Мы хотели бы различать 8%-ное
изменение целевой метрики\
с вероятностью 80%.

### 1. Необходимо выбрать метрику и привести аргументацию.

По условию задачи: "целью этого теста является поиск наилучшего скрипта". \
"наилучший" - понятие относительное. В таком случае я бы смотрел на колонку NPV. \
Однако мы не можем использовать PV (следовательно, NPV тоже) по условию задачи. \
Остается три колонки, с которыми мы можем работать: Флаг дозвона, Флаг продажи, Расходы.

Хотелось бы выделить такую целевую метрику - доля продаж в случае дозвона (все флаги продаж / все влаги дозвона). Однако я не могу придумывать свою метрику из описания 3 пункта ("Определите целевую метрику (= какой-то из колонок) и альтернативу.") - мне необходимо выбрать метрику по одной из 3-х колонок. 

Если считать, что в обоих выборках шанс дозвониться равновероятен, то наиболее подходящей метрикой будет считаться флаг продажи. Если какой-то скрипт сумел продать больше другого, то тогда можно считать, что этот скрипт "лучше"

Итог: целевая метрика - флаг продажи.

### 2. Альтернатива в критерии.

Альтернатива в критерии - двусторонняя. По условию задачи мы хотим определить победителя, поэтому либо есть явный победитель, либо определить победителя невозможно. 

### 3.1. Каков размер выборки? Привести аргументацию и написать как получилось то или иное число.

По формуле для случая Бернулли:\
`N = M = p(1-p)(z_{1-a} - z_{b})^2 / (k * MDE^2)`

Но т.к. альтернатива двусторонняя, то `z_{1-a}` заменяем на `z_{1-a/2}`:\
`N = M = p(1-p)(z_{1-a/2} - z_{b})^2 / (k * MDE^2)`

Где:\
`z_{a}` = `norm.ppf(a)`\
`a = 0.05`\
`b = 0.2`\
`k = 0.5` - считаем, что контроль и тест равен по выборке.\
`p` - конверсия по флагу продаж.\
`MDE = 0.08 * p` - MDE = желаемое изменение метрики * значение метрики на контроле (примерно то же, что и на исторических данных);

In [ ]:
from scipy.stats import norm
import pandas as pd

data = pd.read_csv("hist_telesales.csv", index_col=0)

a = 0.05
b = 0.2
k = 0.5
p = data['Флаг продажи'].mean()
mde = 0.08 * p
z = norm.ppf(1-a/2) - norm.ppf(b)
N = p * (1 - p) * z ** 2 / (k * mde ** 2)

print(f"{mde = }")
print(f"{p = }")
print(f"{N = }")


mde = 0.024167993096174133
p = 0.30209991370217665
N = 5666.310212587125


### 3.2. Необходимо запросить у бота в Телеграме выборку полученного размера командой `/get_project1_sample {размер выборки}` и вписать <ins>код выборки</ins>:

/get_project1_sample 5664

Код выборки: 26329


In [ ]:
control = pd.read_csv("Контроль.csv", index_col=0)
test = pd.read_csv("Тест.csv", index_col=0)

vis_A = len(control)
vis_B = len(test)

conv_A = vis_A * control['Флаг продажи'].mean()
conv_B = vis_B * test['Флаг продажи'].mean()

print(f"{vis_A = }")
print(f"{conv_A = }")
print(f"{vis_B = }")
print(f"{conv_B = }")

vis_A = 5664
conv_A = 1714.0
vis_B = 5664
conv_B = 1682.0


### 4. Принятие решения. Расписать подробно с аргументами.

Проводим тест с помощью сайта https://abtestguide.com/calc/?ua=5664&ub=5664&ca=1714&cb=1682&tail=2

В качестве Confidence берем значение 95%, так как alpha = 0.05. Hypothesis - Two-sided Размеры выборок A (контроль) и B (тест). Обе равны 5664, флагов продажи для A = 1714, для B = 1682.

Не получилось отвергнуть нулевую гипотезу, нет оснований считать выборки разными.

p-value критерия: 0.512.

------------------

##  <ins>Задание 2.</ins>

### 0. Запосить у бота в Телеграме условие командой `/get_project2_desc`.

###Контекст
Мы провели Cust Dev и поняли,
что форма авторизации для клиента сложна,
и из-за этого часть клиентов
отваливается в заявочном процессе.
Мы решили разработать
новый процесс авторизации,
и у нас есть гипотеза,
что он позволит повысить
утилизации наших счетов.
        
###Тест
Мы проводим тест,
в рамках которого мы разбиваем весь поток
на две части рановероятно,
где у одной части (контроля) 
старый процесс авторизации,
а у второй части (теста) - новый.
Целью этого теста
является повышение утилизаций счёта.

###Статистические вводные
Проверка однородности выбранного параметра
осуществяется с уровнем значимости 2%.

Уровень значимости 5%.
Мы хотели бы различать 3%-ное
изменение целевой метрики
с вероятностью 95%.

##Описание данных

Описание колонок в исторических данных.


**ID** - номер заявки, прошедшей скоринг

**Возраст** - возраст клиента, указанный при оформлении КК

**Доход клиента** - доход клиента, указанный при оформлении КК

**Вероятность банкротства** - оценка вероятности банкротства клиента, полученная с помощью скоринга по параметрам клиента

**Флаг утилизация счёта** - совершение первой расходной операции на счёте КК

**Расходы** - сумма затрат на заявку

**PV КК** - 0 если счёт не утилизирован, иначе = PV кредитной карты

**PV услуги** - 0 если счёт не утилизирован, иначе = PV этой услуги

**NPV** = PV КК + PV услуги - Расходы

### 1. Необходимо выбрать метрику и привести аргументацию.

Целью этого теста является повышение утилизаций счёта.

Очевидно, что целевая метрика: Флаг утилизации счёта.

### 2. Альтернатива в критерии.

Альтернатива - односторонняя. Либо тест успешен по сравнению с контролем, либо - провален.



### 3. Нужно выбрать параметр(-ы), влияющий(-ие) на целевую метрику. Привести аргументацию.

Необходимо выбрать параметр, который имеет наибольшее влияние на целевую метрику. Данный параметр должен относиться к социально-демографическим сведениям о клиенте, которые доступны для анализа до включения его в заявку. Рассчитаем корреляцию относительно целевой метрики.

Доступны на выбор:
- Возраст,
- Доход клиента,
- Вероятность банкротства

Посмотрим на их корреляцию с NPV. Возьмем тот, у которого значение корелляции по модулю больше.



In [ ]:
import pandas as pd
data = pd.read_csv("hist_credit_card.csv", index_col=0)
data.corr()['Флаг утилизации счёта'].head(3)

Возраст                    0.007277
Доход клиента              0.000706
Вероятность банкротства   -0.215398
Name: Флаг утилизации счёта, dtype: float64

Выбираем **вероятность банкротства**.

### 4.1. Каков размер выборки? Привести аргументацию и написать как получилось то или иное число.

По формуле для общего случая:\
`N = M = sigma^2*(z_{1-a} - z_{b})^2 / (k * MDE^2)`

Где:\
`z_{a}` = `norm.ppf(a)`\
`a = 0.05`\
`b = 0.05`\
`k = 0.5` - считаем, что контроль и тест равен по выборке.\
`sigma^2` - дисперсия.\
`MDE = 0.03 * p` - MDE = желаемое изменение метрики * значение метрики на контроле (примерно то же, что и на исторических данных);

In [ ]:
from scipy.stats import norm
import pandas as pd

data = pd.read_csv("hist_credit_card.csv", index_col=0)

a = 0.05
b = 0.05
k = 0.5
sigma2 = data['Флаг утилизации счёта'].var()
mde = 0.03 * data['Флаг утилизации счёта'].mean()
z = norm.ppf(1-a) - norm.ppf(b)
N = sigma2 * z ** 2 / (k * mde ** 2)
print(f"{sigma2 = }")
print(f"{sigma2 = }")
print(f"{N = }")

sigma2 = 0.19165807345507177
N = 8382.147778752884


In [ ]:
data['Флаг утилизации счёта'].mean()

0.7415439535921463

### 4.2. Необходимо запросить у бота в Телеграме выборку полученного размера командой `/get_project2_sample {размер выборки}` и вписать <ins>код выборки</ins>:

`/get_project2_sample 8382`

Код выборки: `42705`

### 5. Проверка на однородность, применение критерия. Принятие решения. Расписать подробно с аргументами.

In [ ]:
from statsmodels.stats.weightstats import ztest
from scipy.stats import mannwhitneyu
import numpy as np

control = pd.read_csv("Контроль.csv", index_col=0)
test = pd.read_csv("Тест.csv", index_col=0)

alpha = 0.02
x = control["Вероятность банкротства"]
y = test["Вероятность банкротства"]
print(f"{control['Вероятность банкротства'].mean() = }")
print(f"{test['Вероятность банкротства'].mean() = }")
print(f"{control['Флаг утилизации счёта'].mean() = }")
print(f"{test['Флаг утилизации счёта'].mean() = }")
print(ztest(x, y))
print("Недостаточно доказательств неоднородности. Можно проверять целевую метрику" if ztest(x, y)[1] > alpha else "Точно неоднородна. Тест провален.")

alpha = 0.05
A = control["Флаг утилизации счёта"]
B = test["Флаг утилизации счёта"]

alternative = 'greater'
print(mannwhitneyu(B, A, alternative=alternative))  # H1 - первый аргумент, H0 - второй аргумент
print("Тест провален" if mannwhitneyu(B, A, alternative=alternative)[1] > alpha else "Тест успешен")


control['Вероятность банкротства'].mean() = 0.12439534489854154
test['Вероятность банкротства'].mean() = 0.13021066701188486
control['Флаг утилизации счёта'].mean() = 0.7451682176091625
test['Флаг утилизации счёта'].mean() = 0.7573371510379384
(-3.3583219018346817, 0.0007841723530155859)
Точно неоднородна. Тест провален.
MannwhitneyuResult(statistic=35556444.0, pvalue=0.034202897245221416)
Тест успешен
